[DIY Covid-19 Dashboard Kit](https://github.com/fsmeraldi/diy-covid19dash) (C) Fabrizio Smeraldi, 2020 ([f.smeraldi@qmul.ac.uk](mailto:f.smeraldi@qmul.ac.uk) - [web](http://www.eecs.qmul.ac.uk/~fabri/)). This notebook is released under the [GNU GPLv3.0 or later](https://www.gnu.org/licenses/).

# DIY Covid-19 Dashboard

This page displays covid-19 data in interactive graph format. It is displayed using voila, a python dashboarding tool. 

The following steps were followed to set up the dashbaord:
 - Imported the raw data using the UK Gov's Covid-19 API
 - Assigned the raw data to a json file
 - Wrangled the data and consolidated it into a dataframe
 - Configured widgets using ipywidgets and plotted the graph using matplotlib

The data shown in the graph initially is from a saved json file, however there is an option to refresh the graph using data pulled directly with the API. 

In [15]:
# Import required libraries
from IPython.display import clear_output
import ipywidgets as wdg
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
from uk_covid19 import Cov19API

In [16]:
%matplotlib inline
# Make figures larger
plt.rcParams['figure.dpi'] = 100

In [17]:
# Load the JSON file on dashboard start and assign the raw data as a dictionary in the jsondata variable
with open("vaxvdeath.json", "rt") as INFILE:
    data=json.load(INFILE)
jsondata={'vaxvdeath':data}
#print(jsondata['vaxvdeath'])
#print(jsondata)

In [18]:
def wrangle_data(rawdata):
# This function wrangles the data and returns a dataframe
    datalist=(rawdata['vaxvdeath'])['data']
    #datalist
    
    dates=[dictionary['date'] for dictionary in datalist ]
    dates.sort()
    
    def parse_date(datestring):
        # Convert a date string into a pandas datetime object
        return pd.to_datetime(datestring, format="%Y-%m-%d")

    # Set the startdate of the index to the first date in the data, and the enddate to the last
    startdate=parse_date(dates[0])
    enddate=parse_date(dates[-1])
    index=pd.date_range(startdate, enddate, freq='D')
    #Create the dataframe
    df1=pd.DataFrame(index=index, columns=['cases', 'firstdose', 'seconddose', 'deaths'])

    # Populate the dataframe
    for entry in datalist:
        date=parse_date(entry['date'])
        for column in ['cases', 'firstdose', 'seconddose', 'deaths']:
            if pd.isna(df1.loc[date, column]): 
                #Replace any 'None' values with 0
                value= float(entry[column]) if entry[column]!=None else 0.0
                df1.loc[date, column]=value
                
    #Replace any missing values with 0.0.
    df1.fillna(0.0, inplace=True)
    return df1

# Call the wrangle_data function on the jsondata dictionary
vaxvdeathdf=wrangle_data(jsondata)

## Downloading current data

This button calls the Covid-19 API so that our graph can display the most up-to-date data. Our initial data (from the json file) only contains data from 2020. This is visible in the graph. If you click this button the graph should be updated with data from more recent years. 

In [19]:
def access_api():
# This function calls the API (with a different filter) and returns the downloaded data
    """ Accesses the PHE API. Return data as a like-for-like replacement for the "canned" data loaded from the JSON file. """
    filters = [
        'areaType=nation',
        'areaName=England'
    ]
    structure = {
        "date": "date",
        "cases": "newCasesByPublishDate",
        "firstdose": "newPeopleVaccinatedFirstDoseByPublishDate",
        "seconddose": "newPeopleVaccinatedSecondDoseByPublishDate",
        "deaths":"newDailyNsoDeathsByDeathDate"
    }
    api = Cov19API(filters=filters, structure=structure)
    # Parse the incoming json data and return it
    VDdata=api.get_json()
    return {'vaxvdeath':VDdata}

In [20]:
def api_button_callback(button):
# This function gets fresh data from the API, wrangles the data, then overwrites the dataframe
    # Refer to the access_api function to pull data
    apidata=access_api()
    # Refer to the wrangle_data function to clean up data, then assign the result to the global vaxvdeath dataframe
    global vaxvdeathdf
    vaxvdeathdf=wrangle_data(apidata)
    # Refer to the refresh_graph function (see the graphing code in the next section) to reload the graph with new data
    refresh_graph()
    # Change the button icon to a star
    apibutton.icon="star-o"
    # Disable the button post-refresh
    apibutton.disabled=True

# Our button widget   
apibutton=wdg.Button(
    description="Click to refresh!", 
    disabled=False,
    button_style='success',
    tooltip="Keep calm and carry on",
    icon='bomb'
)

# Register the button callback function with the button
apibutton.on_click(api_button_callback)
# Display the button
display(apibutton)

Button(button_style='success', description='Click to refresh!', icon='bomb', style=ButtonStyle(), tooltip='Kee…

## Graph and Analysis

Behold - our graph!

This graph compares the number of covid-19 cases, first and second vaccine doses, and deaths. The metrics can be viewed individually via the dropdown, and you can also switch between a logarithmic and linear graph, and choose a colour scheme according to your aesthetic preferences.

Though the number of deaths look low compared to the vaccine and case numbers, if we look just at the deaths metric in the graph we can see that there were two large spikes between January 2020 and July 2021, with very small peaks during the period afterwards (but never as high as those first two peaks). Switching to the first dose and second dose metrics, we can see that these were primarily administered during the July 2021-September 2021 period. This could suggest the effectiveness of the vaccines against the virus. At the same time, government guidance on how covid-19 related deaths were reported was changed at the beginning of this year. For further info on this see [here](https://ukhsa.blog.gov.uk/2023/01/27/changes-to-the-way-we-report-on-covid-19-deaths/).

The cases data is slightly inconsistent in that the data initially starts off as being updated every day, but appears after a point to only be updated every week or so (hence the graph looking like it is filled in). Regardless, we can see that the number of cases increases dramatically from July 2021 onwards (opposite to the number of deaths). Could there have been more prevalent testing going on? Or could the new variants have been more contagious?

In [14]:
# Our widgets!
# This widget lets the user select the metric they want to view
series=wdg.SelectMultiple(
    options=['cases', 'firstdose', 'seconddose', 'deaths'],
    value=['cases', 'firstdose', 'seconddose',  'deaths'],
    rows=4,
    description='Metrics:',
    disabled=False
)
# This widget lets the user choose between a linear graph and a log graph
scale=wdg.RadioButtons(
    options=['linear', 'log'],
    value='linear',
    description='Scale:',
    disabled=False
)
# This widget lets the user choose a colour scheme
colors=wdg.ToggleButtons(
    options=['rainbow', 'dark', 'pastel'],
    description='Colormap:',
    disabled=False,
    button_style='info',
    tooltips=['Rainbow colormap', 'Dark colormap', 'Pastel colormap'],
)
# List the first two widgets together horizontally, and the third underneath
controls1=wdg.HBox([series, scale])
controls2=wdg.VBox([colors])


def vaxvdeath_graph(gcols, gscale, gcolor):
# This function plots the graph
    if gscale=='linear':
        logscale=False
    else:
        logscale=True
    ncols=len(gcols)
    if gcolor=='rainbow':
        cm='rainbow'
    elif gcolor=='dark':
        cm='Dark2'
    else:
        cm='Set3'
    if ncols>0:
        vaxvdeathdf[list(gcols)].plot(logy=logscale, title='A comparison of daily cases, vaccine first doses, and deaths', xlabel='date', ylabel='number',colormap=cm)
        plt.show()
    else:
        print("Click to select data for graph")
        print("(Cmd-Click to select more than one category)")

def refresh_graph():
# This function reloads the graph by changing the default value of the scale widget.
# It is used to force a redraw of the graph with new data under the api_button_callback function.  
    current=scale.value
    if current==scale.options[0]:
        other=scale.options[1]
    else:
        other=scale.options[0]
    scale.value=other
    scale.value=current
    
#  Link the widget outputs with the plot_graph function inputs
graph=wdg.interactive_output(vaxvdeath_graph, {'gcols': series, 'gscale': scale, 'gcolor': colors})

# Display the controls and graph
display(controls1, controls2, graph)

Output()

**(C) 2023 Amy Czepliewicz (a.y.czepliewicz@se23.qmul.ac.uk)**
Based on UK Government [data](https://coronavirus.data.gov.uk/) published by [Public Health England](https://www.gov.uk/government/organisations/public-health-england) and on the [DIY Covid Dashboard Kit](https://github.com/fsmeraldi/diy-covid19dash) by Fabrizio Smeraldi. Released under the [GNU GPLv3.0 or later](https://www.gnu.org/licenses/).